In [4]:
import pandas as pd
melb_df = pd.read_csv('data/melb_data_fe.csv')

In [5]:
m0 = melb_df.copy()
m1 = melb_df.copy()
m2 = melb_df.copy()
m0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Suburb           13580 non-null  object 
 1   Rooms            13580 non-null  int64  
 2   Type             13580 non-null  object 
 3   Price            13580 non-null  float64
 4   Method           13580 non-null  object 
 5   SellerG          13580 non-null  object 
 6   Date             13580 non-null  object 
 7   Distance         13580 non-null  float64
 8   Postcode         13580 non-null  int64  
 9   Bedroom          13580 non-null  int64  
 10  Bathroom         13580 non-null  int64  
 11  Car              13580 non-null  int64  
 12  Landsize         13580 non-null  float64
 13  BuildingArea     13580 non-null  float64
 14  CouncilArea      12211 non-null  object 
 15  Lattitude        13580 non-null  float64
 16  Longtitude       13580 non-null  float64
 17  Regionname  

Задание 1.1

Преобразуйте столбец Date в формат datetime и выделите квартал (quarter) продажи объектов недвижимости. Найдите второй по популярности квартал продажи. В качестве ответа запишите число объектов, проданных в этом квартале.

In [6]:
m1.Date = pd.to_datetime(m0.Date)
m1.Date.dt.quarter.value_counts()

3    4873
2    4359
4    2329
1    2019
Name: Date, dtype: int64

Задание 1.2

Преобразуйте все столбцы, в которых меньше 150 уникальных значений, в тип данных category, исключив из преобразования столбцы Date, Rooms, Bedroom, Bathroom, Car.
В качестве ответа запишите результирующее количество столбцов, которые имеют тип данных category.

In [7]:
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        m1[col] = melb_df[col].astype('category') # преобразуем тип столбца

n = len(m1.select_dtypes(include=['category']).columns.tolist())
s = f'Количество столбцов, которые имеют тип данных category: {n}'
s

'Количество столбцов, которые имеют тип данных category: 10'

Задание 2.2

Произведите сортировку столбца AreaRatio по убыванию. При этом индексы полученной таблицы замените на новые. Какое значение площади здания находится в строке 1558? Ответ округлите до целого числа.


In [8]:
dd = m1.sort_values(by='AreaRatio', ascending=False, ignore_index=True).loc[:,['AreaRatio','BuildingArea','Landsize','CouncilArea']]
dd.loc[1558]

AreaRatio             1.0
BuildingArea        126.0
Landsize              0.0
CouncilArea     Glen Eira
Name: 1558, dtype: object

Задание 2.3

Найдите таунхаусы (Type) с количеством жилых комнат (Rooms) больше 2. Отсортируйте полученную таблицу сначала по возрастанию числа комнат, а затем по убыванию средней площади комнат (MeanRoomsSquare). Индексы таблицы замените на новые. Какая цена будет у объекта в строке 18? Ответ запишите в виде целого числа.

In [42]:
m0.head()
mask1 = m0.Rooms > 2
mask2 = m0.Type == 'townhouse'
m0[mask1 & mask2].sort_values(
    by=['Rooms', 'MeanRoomsSquare'], 
    ascending=[True, False], 
    ignore_index=True
    ).loc[18:18,['Rooms','MeanRoomsSquare','Price']]

,Rooms,MeanRoomsSquare,Price
18,3,27.75,1300000.0


In [49]:
melb_df.groupby('Regionname')['SellerG'].agg(
    		['nunique', set]
).sort_values(by='nunique')

,nunique,set
Regionname,,
Western Victoria,6,"{hockingstuart, HAR, other, YPA, Ray, Raine}"
Eastern Victoria,11,"{Harcourts, hockingstuart, HAR, McGrath, other..."
Northern Victoria,11,"{McDonald, hockingstuart, HAR, McGrath, other,..."
South-Eastern Metropolitan,25,"{Thomson, Biggin, RW, Hodges, Buxton, Eview, H..."
Eastern Metropolitan,26,"{Kay, Love, Biggin, RW, Buxton, Harcourts, McG..."
Western Metropolitan,34,"{Alexkarbon, Biggin, Bells, RW, Hodges, Harcou..."
Southern Metropolitan,38,"{Thomson, Kay, Biggin, RW, Hodges, Buxton, Evi..."
Northern Metropolitan,40,"{Kay, Love, Alexkarbon, Biggin, RW, Hodges, Ev..."


Задание 3.1

Сгруппируйте данные по признаку количества комнат и найдите среднюю цену объектов недвижимости в каждой группе. В качестве ответа запишите количество комнат, для которых средняя цена наибольшая.

In [54]:
melb_df.groupby('Rooms')['Price'].agg(
    		['mean']
).sort_values(by='mean')

,mean
Rooms,
1,4.338245e+05
2,7.750812e+05
10,9.000000e+05
3,1.076081e+06
4,1.445282e+06
8,1.602750e+06
6,1.849366e+06
5,1.870260e+06
7,1.920700e+06


Задание 3.2

Какой регион имеет наименьшую протяжённость по географической широте (Lattitude)?
Для ответа на этот вопрос рассчитайте стандартное отклонение широты для каждого региона.
В качестве ответа запишите название этого региона.

In [58]:

melb_df.groupby('Regionname')['Lattitude'].agg(['std']).sort_values(by='std')

,std
Regionname,
Western Victoria,0.011579
Southern Metropolitan,0.043080
Eastern Metropolitan,0.047890
Northern Metropolitan,0.049639
Western Metropolitan,0.051251
South-Eastern Metropolitan,0.073411
Northern Victoria,0.084455
Eastern Victoria,0.147067


Задание 3.3

Какая риелторская компания (SellerG) имеет наименьшую общую выручку за период с 1 мая по 1 сентября (включительно) 2017 года?
Для ответа на этот вопрос рассчитайте сумму продаж (Price) каждой компании в заданный период.
Не забудьте перевести даты в формат datetime.

In [71]:
m1.Date = pd.to_datetime(m0.Date)
# m1.Date.describe()
# m1.Date.info()
mask1 = m1.Date >= pd.to_datetime('01.05.2017')
mask2 = m1.Date <= pd.to_datetime('01.09.2017')
m1[mask1].groupby('SellerG')['Price'].agg(['sum']).sort_values(by='sum')[0:3]

,sum
SellerG,
LITTLE,6429500.0
Burnham,10173000.0
Thomson,12967000.0


Задание 4.2

Составьте сводную таблицу, которая показывает зависимость медианной площади (BuildingArea) здания от типа объекта недвижимости (Type) и количества жилых комнат в доме (Rooms). Для какой комбинации признаков площадь здания наибольшая?
В качестве ответа запишите эту комбинацию (тип здания, число комнат) через запятую, без пробелов.

Задание 4.3

Составьте сводную таблицу, которая показывает зависимость средней цены объекта недвижимости (Price) от риелторского агентства (SellerG) и типа здания (Type).
Во вновь созданной таблице найдите агентство, у которого средняя цена для зданий типа unit максимальна. В качестве ответа запишите название этого агентства.

In [113]:
# melb_df.pivot_table(
#     values='BuildingArea',
#     index='Rooms',
#     columns='Type',
#     aggfunc='median',
#     fill_value=0
# )
# mm#.max().max()

pivot = melb_df.pivot_table(
    values='Price',
    index='SellerG',
    columns='Type',
    # aggfunc='median',
    aggfunc='mean',
    fill_value=0
)
# pivot.columns
# pivot['unit'].max()  #993642.85
p_unit  = pivot['unit']
p_unit[p_unit > 990000]


SellerG
Nick    993642.857143
Name: unit, dtype: float64